План семинара **"Реализация собственных операторов на языке Python в фреймворке Pytorch"**
1. [Squeeze-and-Excitation (SE) Block](https://arxiv.org/abs/1709.01507)
2. [Selective Kernel (SK) Convolution](https://arxiv.org/abs/1903.06586)

# Squeeze-and-Excitation (SE) Block

“Squeeze-and-Excitation” (SE) block can adaptively recalibrates
channel-wise feature responses by explicitly modelling interdependencies between channels. 

In [ ]:
import torch
from torch import nn

class SEBlock(nn.Module):
    """
    Implementation of the Squeeze-and-Excitation (SE) block proposed in [1].
    Parameters
    ----------
    in_channels : int
        Number of channels in the input tensor.
    reduction : int, optional, default=16
        Reduction ratio to control the intermediate channel dimension.
    References
    ----------
    1. "`Squeeze-and-Excitation Networks. <https://arxiv.org/abs/1709.01507>`_" Jie Hu, et al. CVPR 2018.
    """

    def __init__(
        self,
        in_channels: int,
        reduction: int = 16
    ) -> None:
        super(SEBlock, self).__init__()

        out_channels = ...
        self.squeeze = ...
        self.excitation = nn.Sequential(
            ...)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Parameters
        ----------
        x : torch.Tensor (batch_size, in_channels, height, width)
            Input tensor.
        Returns
        -------
        out : torch.Tensor (batch_size, in_channels, height, width)
            Output of the SK convolution layer.
        """
        # x: [b, c, h, w]

        z = ... # eq.2 [b, c, 1, 1]
        s = ... # eq.3 [b, c, 1, 1]
        out =  ... # eq. 4 [b, c, h, w]
        return out

In [ ]:
features = torch.rand(1, 32, 25, 25)
out = SEBlock(32)
out(features).shape

torch.Size([1, 32, 25, 25])

# Selective Kernel (SK) Convolution

To enable the neurons to adaptively adjust their RF sizes,
we propose an automatic selection operation, “Selective
Kernel” (SK) convolution, among multiple kernels with different kernel sizes

In [ ]:
import torch
from torch import nn
from typing import List, Optional

class SKConv(nn.Module):
    """
    Implementation of the Selective Kernel (SK) Convolution proposed in [1].
    Parameters
    ----------
    in_channels : int
        Number of channels in the input tensor.
    out_channels : int
        Number of channels produced by the convolution.
    kernels : List[int], optional, default=[3, 5]
        List of kernel sizes for each branch.
    reduction : int, optional, default=16
        Reduction ratio to control the dimension of "compact feature" ``z`` (see eq.4).
    L : int, optional, default=32
        Minimal value of the dimension of "compact feature" ``z`` (see eq.4).
    groups : int, optional, default=32
        Hyperparameter for ``torch.nn.Conv2d``.
    References
    ----------
    1. "`Selective Kernel Networks. <https://arxiv.org/abs/1903.06586>`_" Xiang Li, et al. CVPR 2019.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: Optional[int] = None,
        kernels: List[int] = [3, 5],
        reduction: int = 16,
        L: int = 32,
        groups: int = 32
    ) -> None:
        super(SKConv, self).__init__()

        if out_channels is None:
            out_channels = in_channels
        self.out_channels = out_channels

        d = ... # eq.4

        self.M = len(kernels)

        self.convs = nn.ModuleList([
                nn.Sequential(
                  ...
            )
            for k in kernels
        ])

        self.pool = ...

        self.fc_z = nn.Sequential(
            ...)
        self.fc_attn = nn.Linear(...)
        self.softmax = nn.Softmax(...)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Parameters
        ----------
        x : torch.Tensor (batch_size, in_channels, height, width)
            Input tensor.
        Returns
        -------
        out : torch.Tensor (batch_size, out_channels, height, width)
            Output of the SK convolution layer.
        """
        #Conv2d , AvgPoll, softmax, ReLU, BatchNorm, Linear

        # ----- split -----
        # x: [b, c, h, w]
        feats = ...  # [b, M, c, h, w]

        # ----- fuse -----
        # eq.1
        U = ...
        # channel-wise statistics, eq.2
        s = ... #s: [b, c]
        # compact feature, eq.3
        z = ... # z [b, d]

        # ----- select -----
        batch_size, out_channels = s.shape

        # attention map, eq.5
        score = ...  # (batch_size, M * out_channels)
        score = ...)  # (batch_size, M, out_channels, 1, 1)
        att = ...


        # fuse multiple branches, eq.6
        out = ...  # (batch_size, out_channels, height, width)
        return out

In [ ]:
features = torch.rand(1, 34*16, 25, 25)
out = SKConv(34*16).eval()
out(features).shape

torch.Size([1, 544, 25, 25])

In [ ]:
n = nn.Conv2d(3, 3, kernel_size=3)
n.weight.shape

torch.Size([3, 3, 3, 3])

In [ ]:
n = nn.Conv2d(3, 3, kernel_size=3, groups=3)
n.weight.shape

torch.Size([3, 1, 3, 3])

In [ ]:
features = torch.rand(1, 3, 25, 25)
n(features).shape

torch.Size([1, 3, 23, 23])